# pytorch实现神经网络模型

__各种网络模型详解建议看B站up主：霹雳吧啦Wz，用户ID：18161609__   

__B站部分内容__：[ResNet网络结构，BN以及迁移学习详解](https://www.bilibili.com/video/BV1T7411T7wa/?share_source=copy_web&vd_source=af4d80c0a2a9115a896a0378b7093d65)  

**该up主的github也有许多资源**：[霹雳吧啦Wz的github地址](https://github.com/WZMIAOMIAO/deep-learning-for-image-processing)  

**该up的CSDN**：<https://blog.csdn.net/qq_37541097>

### （1）**LeNet**

#### **1.1 简介**

LeNet首次采用了卷积层、池化层这两个全新的神经网络组件，接收灰度图像，并输出其中包含的手写数字，在手写字符识别任务上取得了瞩目的准确率。  
LeNet网络的一系列的版本，以LeNet-5版本最为著名，也是LeNet系列中效果最佳的版本。
  
![LeNet结构](./LeNet结构.png)


#### **1.2 模型特点和模型实现**

- 提出卷积神经网络的基本框架：卷积层、池化层、全连接层。  

- 权重共享，参数更少，减少计算量，降低内存。  

- 卷积层的局部连接，保证图像的空间相关性。  

- 空间均值下采样，减少特征数量。  

- 使用非线性激活函数sigmoid和tanh,使用哪种都行。

In [ ]:
# lenet代码实现

import torch
import torch.nn as nn


class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            # 这里经过池化之后大小是16X5X5，也就是16个通道，每个通道5X5，
            # 再用kernel_size=5的卷积核进行卷积后大小为120X1X1，即120个1X1
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5),
            # nn.Flatten()将指定的维度展平，得到tensor类型的数据，默认是将第1个维度以及之后的
            # 维度展平，注意维度从0开始，比如tensor的维度是(32,1,5,5),经过nn.Flatten()操作后
            # 维度变为(32,1*5*5=25),如果指定维度，如nn.Flatten(0,2)，则维度变为(32*1*5=160,5)
            # 也可以在前向传播函数中使用view()将卷积之后的数据展平，然后才能正确输入全连接层。
            nn.Flatten(),
            nn.Linear(in_features=120, out_features=84),
            nn.Linear(in_features=84, out_features=10),
        )

    def forward(self, x):
        x = self.conv(x)
        return x

In [2]:
model = LeNet5()

# 28X28是mnist数据集中图像的尺寸，输入模型后经过padding=2填充为32X32。
x = torch.rand([1, 1, 28, 28])
y = model(x)
print(y)

tensor([[ 0.0770,  0.0575,  0.1159, -0.0692, -0.0008,  0.0769, -0.0906, -0.0236,
         -0.1010,  0.0644]], grad_fn=<AddmmBackward0>)


#### __1.3 解惑__ 

 - LeNet5的输入明明是1X1X32X32，为什么上面代码要输入`x = torch.rand([1, 1, 28, 28])`？  
**参考**：[LeNet：第一个卷积神经网络](https://www.ruanx.net/lenet/) 。   
因为28X28是mnist数据集中图像的大小，所以完全是为了使用该数据集。输入这样尺寸的数据后，LeNet5模型中的代码：`nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)`就是负责将图片填充到32X32的大小（padding=2，上下左右各加两行之后就变成32X32），填充之后再进行卷积，经过卷积层输出后图像的大小计算公式是：$$(N+2P-K)/S+1$$  
**其中N是图像的长或宽，P是填充的行数padding，K是卷积核尺寸kernel_size，S是stride，即卷积核每次走多少像素。**    
所以输入的28X28经填充后变为32X32，再经过第一次卷积之后变为6通道的28X28图像。

### （2）**AlexNet**    

#### __2.1 结构和特点__

1. **结构**  
**参考**：[卷积神经网络经典回顾之AlexNet-知乎](https://zhuanlan.zhihu.com/p/618545757)  
AlexNet输入为RGB三通道的224 × 224 × 3大小的图像（也可填充为227 × 227 × 3 ）。  
AlexNet 共包含5 个卷积层（包含3个池化）和 3 个全连接层。其中，每个卷积层都包含卷积核、偏置项、ReLU激活函数和局部响应归一化（LRN）模块。  
第1、2、5个卷积层后面都跟着一个最大池化层，后三个层为全连接层。最终输出层为softmax，将网络输出转化为概率值，用于预测图像的类别。  

   ![AlexNet结构](./AlexNet结构.jpg) 

2. **模型特点**  
- **真正意义上的深度卷积神经网络**。能更好地学习特征，提高分类精度。 

- __首次使用ReLU__。相比于传统的 sigmoid 和 tanh 函数，ReLU 能够在保持计算速度的同时，有效解决梯度消失问题，从而使得训练更加高效。  

- __局部响应归一化LRN的使用__。在卷积层中，每个卷积核都对应一个特征图（feature map），LRN就是对这些特征图进行归一化。具体来说，对于每个特征图上的每个位置，计算该位置周围的像素的平方和，然后将当前位置的像素值除以这个和。LRN本质是抑制邻近神经元的响应，从而增强了神经元的较大响应。这种技术在一定程度上能够避免过拟合，并提高网络的泛化能力。  

- **数据增强和Dropout**。数据增强增加训练数据的多样性，提高模型泛化能力。Dropout是在前向传播过程中让神经元以一定的概率停止工作，使模型不会太依赖某些局部特征，提高模型泛化能力。  

- **大规模分布式训练**。AlexNet在使用GPU进行训练时，可将卷积层和全连接层分别放到不同的GPU上进行并行计算，从而大大加快了训练速度。像这种大规模 GPU 集群进行分布式训练的方法在后来的深度学习中也得到了广泛的应用。

3. __模型实现__

In [3]:
# AlexNet模型实现

import torch
import torch.nn as nn


class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, init_weights=False):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(
                3, 48, kernel_size=11, stride=4, padding=2
            ),  # input[3, 224, 224]  output[48, 55, 55]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # output[48, 27, 27]
            nn.Conv2d(48, 128, kernel_size=5, padding=2),  # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # output[128, 13, 13]
            nn.Conv2d(128, 192, kernel_size=3, padding=1),  # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=3, padding=1),  # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 128, kernel_size=3, padding=1),  # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(128 * 6 * 6, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

    # 权重初始化函数
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [4]:
alexnet = AlexNet()
x = torch.rand([1, 3, 224, 224])
y = alexnet(x)  # y是个长度为1000的行向量。
print(y.shape)

torch.Size([1, 1000])


#### __2.2 解惑__  


- 模型定义中哪里体现了局部响应归一化LRN？  
**参考**：[AlexNet详细解读-博客园](https://www.cnblogs.com/xiaoboge/p/10465534.html)  
没有理解LRN是怎么在模型定义中体现的。

### （3）__VGG__

#### __3.1 VGG结构和特点__

参考：[一文读懂VGG-知乎](https://zhuanlan.zhihu.com/p/41423739)  

- __VGG原理__  
VGG主要工作是证明了增加网络的深度能够在一定程度上影响网络最终的性能。VGG有两种结构，分别是VGG16和VGG19，两者并没有本质上的区别，只是网络深度不一样。  
VGG16相比AlexNet的一个改进是采用连续的几个3x3的卷积核代替AlexNet中的较大卷积核（11x11，7x7，5x5）。  
对于给定的感受野（与输出有关的输入图片的局部大小），采用堆积的小卷积核是优于采用大的卷积核，因为多层非线性层可以增加网络深度来保证学习更复杂的模式，而且代价还比较小（参数更少）。
- __VGG结构__  

   ![VGG结构](./VGG结构.png)  
- __VGG优缺点__  
__优点__：结构简洁，整个网络使用同样大小的卷积核尺寸和最大池化尺寸；几个小滤波器的组合大于一个大滤波器；验证了加深网络结构可提升性能。  
__缺点__：耗费资源，参数更多，主要来自第一个全连接层。

#### __3.2 代码实现__

In [5]:
import torch.nn as nn
import torch

# official pretrain weights
model_urls = {
    "vgg11": "https://download.pytorch.org/models/vgg11-bbd30ac9.pth",
    "vgg13": "https://download.pytorch.org/models/vgg13-c768596a.pth",
    "vgg16": "https://download.pytorch.org/models/vgg16-397923af.pth",
    "vgg19": "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth",
}


class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=False):
        super(VGG, self).__init__()
        # 这里特征提取层features（即卷积层）另外使用函数编写。
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        # N x 3 x 224 x 224
        x = self.features(x)
        # N x 512 x 7 x 7
        x = torch.flatten(x, start_dim=1)
        # N x 512*7*7
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                # nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_features(cfg: list):
    """制作特征提取层"""
    layers = []
    in_channels = 3
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    "vgg11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "vgg13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "vgg16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "vgg19": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}


def vgg(model_name="vgg16", **kwargs):
    """生成vgg模型"""
    assert model_name in cfgs, "Warning: model number {} not in cfgs dict!".format(
        model_name
    )
    cfg = cfgs[model_name]

    model = VGG(make_features(cfg), **kwargs)
    return model

In [6]:
vgg16 = vgg(model_name="vgg16")
x = torch.rand([1, 3, 224, 224])
y = vgg16(x)
print(y.shape)

torch.Size([1, 1000])


### （4）__InceptionNet(GoogLeNet)__

#### __4.1 InceptionNet结构和特点__

- __InceptionNet提出背景__  
**参考**：[inception-知乎](https://zhuanlan.zhihu.com/p/73857137)  
一般来说，提高网络性能最直接的办法就是增加深度和宽度，但是一味低增加，会导致参数过多，如果训练数据有限，会导致过拟合；参数多，计算量大，难以应用；网络越深，越容易出现梯度弥散，难以优化模型。  

- __InceptionNet核心思想__  
**参考**：[深入理解GoogLeNet结构（原创）-知乎](https://zhuanlan.zhihu.com/p/32702031)  
inception模块的基本机构如下图，整个inception结构就是由多个这样的inception模块串联起来的。inception结构的主要贡献有两个：一是使用1x1的卷积来进行升降维；二是在多个尺寸上同时进行卷积再聚合。  
  
   ![inception模块结构](./inception模块结构.jpg)  

使用1X1卷积实现降维，可以明显减少参数量，原理如下图。  
**参考**：[深度学习：GoogLeNet结构详解-CSDN](https://blog.csdn.net/Vermont_/article/details/108836111)  

   ![inception1X1降维](./inception1X1降维.png)  

#### **4.2 InceptionNet代码实现**

In [7]:
import torch.nn as nn
import torch
import torch.nn.functional as F


class GoogLeNet(nn.Module):
    def __init__(self, num_classes=1000, aux_logits=True, init_weights=False):
        super(GoogLeNet, self).__init__()
        self.aux_logits = aux_logits

        self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.conv2 = BasicConv2d(64, 64, kernel_size=1)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        # N x 3 x 224 x 224
        x = self.conv1(x)
        # N x 64 x 112 x 112
        x = self.maxpool1(x)
        # N x 64 x 56 x 56
        x = self.conv2(x)
        # N x 64 x 56 x 56
        x = self.conv3(x)
        # N x 192 x 56 x 56
        x = self.maxpool2(x)

        # N x 192 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        x = self.inception3b(x)
        # N x 480 x 28 x 28
        x = self.maxpool3(x)
        # N x 480 x 14 x 14
        x = self.inception4a(x)
        # N x 512 x 14 x 14
        if self.training and self.aux_logits:  # eval model lose this layer
            aux1 = self.aux1(x)

        x = self.inception4b(x)
        # N x 512 x 14 x 14
        x = self.inception4c(x)
        # N x 512 x 14 x 14
        x = self.inception4d(x)
        # N x 528 x 14 x 14
        if self.training and self.aux_logits:  # eval model lose this layer
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        # N x 832 x 14 x 14
        x = self.maxpool4(x)
        # N x 832 x 7 x 7
        x = self.inception5a(x)
        # N x 832 x 7 x 7
        x = self.inception5b(x)
        # N x 1024 x 7 x 7

        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = torch.flatten(x, 1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        # N x 1000 (num_classes)
        if self.training and self.aux_logits:  # eval model lose this layer
            return x, aux2, aux1
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()

        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, kernel_size=1),
            BasicConv2d(
                ch3x3red, ch3x3, kernel_size=3, padding=1
            ),  # 保证输出大小等于输入大小
        )

        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, kernel_size=1),
            BasicConv2d(
                ch5x5red, ch5x5, kernel_size=5, padding=2
            ),  # 保证输出大小等于输入大小
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(in_channels, pool_proj, kernel_size=1),
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)


class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.averagePool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = BasicConv2d(
            in_channels, 128, kernel_size=1
        )  # output[batch, 128, 4, 4]

        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14
        x = self.averagePool(x)
        # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4
        x = self.conv(x)
        # N x 128 x 4 x 4
        x = torch.flatten(x, 1)
        x = F.dropout(x, 0.5, training=self.training)
        # N x 2048
        x = F.relu(self.fc1(x), inplace=True)
        x = F.dropout(x, 0.5, training=self.training)
        # N x 1024
        x = self.fc2(x)
        # N x num_classes
        return x


class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

In [8]:
googleNet = GoogLeNet()
x = torch.rand([1, 3, 224, 224])
y = googleNet(x)
# print(y)

### （5）**ResNet**

#### 5.1 **ResNet结构**

**参考**：[ResNet网络结构详解与模型的搭建](http://t.csdnimg.cn/F7VpE)  

1. **亮点**：
- 提出**residual结构（残差结构）**，并搭建超深的网络结构(突破1000层)  
- 使用**Batch Normalization**加速训练(丢弃dropout)  

ResNet34结构如下：

![ResNet34结构](./ResNet结构.jpeg)

下图是论文中给出的两种残差结构。  
  
![residual结构](./ResNet残差结构.png)  

左边的残差结构是针对层数较少网络，例如ResNet18层和ResNet34层网络。右边是针对网络层数较多的网络，例如ResNet101，ResNet152等。  

为什么深层网络要使用右侧的残差结构呢。因为，右侧的残差结构能够减少网络参数与运算量。同样输入、输出一个channel为256的特征矩阵，如果使用左侧的残差结构需要大约1170648个参数，但如果使用右侧的残差结构只需要69632个参数。明显搭建深层网络时，使用右侧的残差结构更合适。  

2. **捷径分支（shortcut）**：  

显然，主分支上的输出矩阵和捷径分支上的输出矩阵能够相加的前提是两者有相同的shape,所以要注意捷径分支上所使用的1X1卷积的通道数以及卷积的stride。  

ResNet50/101/152的残差结构如下图,该残差结构所对应的虚线残差结构如下图右侧所示，在捷径分支上有一层1x1的卷积层，它的卷积核个数与主分支上的第三层卷积层卷积核个数相同，注意每个卷积层的步距。  
（注意：原论文中，在下图右侧虚线残差结构的主分支中，第一个1x1卷积层的步距是2，第二个3x3卷积层步距是1。但在pytorch官方实现过程中是第一个1x1卷积层的步距是1，第二个3x3卷积层步距是2，这么做的好处是能够在top1上提升大概0.5%的准确率。可参考[Resnet v1.5](https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch) 

![虚线残差结构](./虚线残差结构.png) 


**不同深度的ResNet网络结构配置如下图**：  

![ResNet结构配置](./不同深度的ResNet结构配置.png)  

对于ResNet18/34/50/101/152，表中conv3_x, conv4_x, conv5_x所对应的一系列残差结构的第一层残差结构都是虚线残差结构。因为这一系列残差结构的第一层都有调整输入特征矩阵shape的使命（将特征矩阵的高和宽缩减为原来的一半，将深度channel调整成下一层残差结构所需要的channel）。


#### 5.2 **ResNet代码实现**